In [28]:
import pandas as pd

# Load the Excel file
file_path = "b1_modified.xls"
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame
df.head(10)


,Módulo,Funcionalidad,Profesional,Limitación CRM,Limitación logística,Limitación financiera,B1 Starter User
0,Gestión,Crear nueva sociedad,Completa,NaN,NaN,NaN,Completa
1,NaN,Seleccionar sociedad,Completa,Completa,Completa,Completa,Completa
2,NaN,Tipo de cambio e índices de precios,Completa,NaN,NaN,Completa,Completa
3,NaN,Inicialización del sistema,Completa,NaN,NaN,NaN,Completa
4,NaN,Configuración general,Completa,NaN,NaN,NaN,Completa
5,NaN,Cambio de clave de acceso,Completa,Completa,Completa,Completa,Completa
6,NaN,Parametrizaciones de finanzas,Completa,NaN,NaN,NaN,Completa
7,NaN,Parametrización de oportunidades,Completa,NaN,NaN,NaN,Completa
8,NaN,Parametrización de compras,Completa,NaN,NaN,NaN,Completa
9,NaN,Parametrizaciones de IC,Completa,NaN,NaN,NaN,Completa


In [29]:
# Forward fill the empty values in the "Módulo" column
df['Módulo'] = df['Módulo'].fillna(method='ffill')

# Define a function to transform permission values
def transform_permissions(value):
    if pd.isna(value):
        return "None"
    elif "Completa" in value:
        return "All"
    else:
        return "Limited"

# Apply the transformation function to the permission columns
permission_cols = ["Profesional", "Limitación CRM", "Limitación logística", "Limitación financiera", "B1 Starter User"]
for col in permission_cols:
    df[col] = df[col].apply(transform_permissions)

# Remove leading and trailing whitespaces from column names
df.columns = df.columns.str.strip()

# Remove leading and trailing whitespaces from 'Funcionalidad' entries
df['Funcionalidad'] = df['Funcionalidad'].str.strip()

# Display the first few rows of the updated DataFrame
df.head()

,Módulo,Funcionalidad,Profesional,Limitación CRM,Limitación logística,Limitación financiera,B1 Starter User
0,Gestión,Crear nueva sociedad,All,None,None,None,All
1,Gestión,Seleccionar sociedad,All,All,All,All,All
2,Gestión,Tipo de cambio e índices de precios,All,None,None,All,All
3,Gestión,Inicialización del sistema,All,None,None,None,All
4,Gestión,Configuración general,All,None,None,None,All


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class QueryProgram:
    def __init__(self, df):
        self.df = df
        self.vectorizer = TfidfVectorizer()

        # Apply the vectorizer to the Funcionalidad column
        self.tfidf_matrix = self.vectorizer.fit_transform(df['Funcionalidad'])

    def query(self, prompt, top_n=5):
        # Vectorize the prompt
        prompt_vec = self.vectorizer.transform([prompt])

        # Calculate cosine similarity between the prompt and functionalities
        cos_sim = cosine_similarity(prompt_vec, self.tfidf_matrix).flatten()

        # Get indices of the top_n most similar functionalities
        top_indices = cos_sim.argsort()[-top_n:][::-1]

        # Extract the top_n most similar functionalities from the DataFrame
        similar_funcionalidades = self.df.iloc[top_indices]

        # Define a function to rank functionalities based on permissions
        def rank(row):
            score = 0
            if row["Limitación CRM"] == "All":
                score += 3
            if row["Limitación logística"] == "All":
                score += 2
            if row["Limitación financiera"] == "All":
                score += 1
            return score

        # Apply the ranking function to the DataFrame
        similar_funcionalidades["Rank"] = similar_funcionalidades.apply(rank, axis=1)

        # Sort the DataFrame by the ranking
        similar_funcionalidades = similar_funcionalidades.sort_values(by="Rank", ascending=False)

        return similar_funcionalidades

# Initialize the QueryProgram with the DataFrame
query_program = QueryProgram(df)

# Test the program with a prompt
test_prompt = "facturas"
test_results = query_program.query(test_prompt)

test_results

/var/folders/fd/542lzl1n1lb8n5gjc83hngyh0000gn/T/ipykernel_15633/3774131286.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_funcionalidades["Rank"] = similar_funcionalidades.apply(rank, axis=1)


,Módulo,Funcionalidad,Profesional,Limitación CRM,Limitación logística,Limitación financiera,B1 Starter User,Rank
59,Ventas: deudores,Impresión de documentos,All,All,All,All,All,6
61,Ventas: deudores,Informes y cuadros de ventas,All,All,All,None,All,5
58,Ventas: deudores,Transacciones recurrentes (Modelos),All,All,All,Limited,All,5
57,Ventas: deudores,Asistente para la creación dedocumentos,All,None,All,None,All,2
166,Movilidad,Aplicación móvil para iPhone y iPad,All,Limited,Limited,Limited,Limited,0


In [31]:
# Define weights for each permission level
weights = {"All": 3, "Limited": 2, "None": 1}

def rank_permissions(df):
    # Initialize a dictionary to store the scores
    scores = {}

    # For each permission column...
    for col in permission_cols:
        # ...count the number of "All", "None", and "Limited" instances...
        counts = df[col].value_counts()
        
        # ...and calculate the score for this permission
        score = sum(weights[level] * count for level, count in counts.items())
        
        # Store the score in the dictionary
        scores[col] = score

    # Convert the dictionary to a DataFrame and sort it by the scores
    scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Score'])
    scores_df = scores_df.sort_values(by='Score', ascending=False)
    
    return scores_df

# Rank the permissions for the test results
ranked_permissions = rank_permissions(test_results)
ranked_permissions


,Score
Profesional,15
Limitación logística,14
B1 Starter User,14
Limitación CRM,12
Limitación financiera,9
